## Rapport TME1 - Arbres de décision, sélection de modèles

Membres du binôme :
- KRISNI Almehdi
- ARICHANDRA Santhos

Dans ce TME, on aborde les aspects essentiels des arbres de décision. Les arbres sont des modèles de classification hiérarchique, en d'autres termes, pour des exemples de la forme **x** = *(x1, x2, ..., xd)*, on associe à chaque noeud de l'arbre un test sur une des dimensions de *xi* de la forme *xi* <= s avec s une valeur réelle. Le test indique le noeud fils devant être sélectionné. La feuille atteinte après l'ensemble des tests est la classe prédite pour un exemple.

On cherche à implémenter les fonctions utiles au calcul du partitionnement optimal, soit les fonctions de calcul d'entropie. Toutes les méthodes sont situées dans le fichier *entropie.py* du répertoire TME1.

In [8]:
# Import du fichier entropie.py
import entropie as ent

# Import utilitaires
import random
import numpy as np
import pickle

### Question 1
On code la méthode *entropie* permettant à partir d'un vecteur (une liste ou un vecteur numpy), de calculer l'entropie de ce vecteur. Une entropie importante représente un ensemble de données hétérogène tandis qu'une entropie nulle représente un ensemble homogéne, soit composé d'une seule et unique valeur.

On réalise quelques tests avec la méthode crée sur des vecteurs différents.

In [9]:
# Tests d'entropie
vT1 = [random.randint(0, 10) for _ in range(10)]
vT2 = [random.randint(0, 10) for _ in range(10)]
vT3 = [1 for _ in range(10)]

print("Vecteur :", vT1, "\tEntropie :", ent.entropie(vT1))
print("Vecteur :", vT2, "\tEntropie :", ent.entropie(vT2))
print("Vecteur :", vT3, "\tEntropie :", ent.entropie(vT3))

Vecteur : [2, 0, 2, 5, 8, 4, 4, 4, 9, 1] 	Entropie : 1.8343719702816235
Vecteur : [6, 1, 8, 5, 1, 6, 6, 10, 9, 9] 	Entropie : 1.6957425341696346
Vecteur : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 	Entropie : 0.0


### Question 2

On code la méthode *entropie_cond* qui à partir d'une liste de vecteurs, calcule l'homogéinité de la partition obtenue. Il s'agit de la moyenne pondérée des entropies des sous-ensembles obtenus.

On réalise un test avec les vecteurs créés à l'étape précédente.

In [10]:
# Test d'entropie conditionnelle
print("Ensemble de vecteurs :", vT1, vT2, vT3, sep="\n")
print("\nEntropie conditionnelle de la partition :", ent.entropie_cond([vT1, vT2, vT3]))

Ensemble de vecteurs :
[2, 0, 2, 5, 8, 4, 4, 4, 9, 1]
[6, 1, 8, 5, 1, 6, 6, 10, 9, 9]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Entropie conditionnelle de la partition : 1.176704834817086


### Question 3

Afin de charger un extrait des données de la base imdb, on dispose d'un code fourni par le sujet du TME permettant le chargement d'une zone précise du fichier. On exécute le code afin de conserver en mémoire les données et ne pas avoir à les recharger à chaque utilisation.

In [11]:
# Data : tableau (films, features), id2titles : dictionnaire id -> titre,
# fields : id feature -> nom
[data, id2titles, fields]= pickle.load(open("imdb_extrait.pkl", "rb"))
# La derniere colonne est le vote
datax = data [:,:32]
datay = np.array([1 if x [33] > 6.5 else -1 for x in data ])